In [87]:
import numpy as np
import pandas as pd
import sqlalchemy

In [88]:
df = pd.read_csv('C:/Users/CVR/Downloads/Automobile.csv')
df

,symboling,normalized_losses,make,fuel_type,aspiration,number_of_doors,body_style,drive_wheels,engine_location,wheel_base,...,engine_size,fuel_system,bore,stroke,compression_ratio,horsepower,peak_rpm,city_mpg,highway_mpg,price
0,3,168,alfa-romero,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495
1,3,168,alfa-romero,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500
2,1,168,alfa-romero,gas,std,two,hatchback,rwd,front,94.5,...,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500
3,2,164,audi,gas,std,four,sedan,fwd,front,99.8,...,109,mpfi,3.19,3.40,10.0,102,5500,24,30,13950
4,2,164,audi,gas,std,four,sedan,4wd,front,99.4,...,136,mpfi,3.19,3.40,8.0,115,5500,18,22,17450
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196,-1,95,volvo,gas,std,four,sedan,rwd,front,109.1,...,141,mpfi,3.78,3.15,9.5,114,5400,23,28,16845
197,-1,95,volvo,gas,turbo,four,sedan,rwd,front,109.1,...,141,mpfi,3.78,3.15,8.7,160,5300,19,25,19045
198,-1,95,volvo,gas,std,four,sedan,rwd,front,109.1,...,173,mpfi,3.58,2.87,8.8,134,5500,18,23,21485
199,-1,95,volvo,diesel,turbo,four,sedan,rwd,front,109.1,...,145,idi,3.01,3.40,23.0,106,4800,26,27,22470


In [89]:
# 1. Handling Missing Values

# Identify missing values in normalized_losses, bore, stroke, horsepower,
# peak_rpm, price.
missing_values = df[['normalized_losses', 'bore', 'stroke', 'horsepower', 'peak_rpm', 'price']].isnull().sum()
print(missing_values)

# Fill numerical missing values using:
# Mean/Median for horsepower, price.
df['horsepower'].fillna(df['horsepower'].mean(), inplace=True)
df['price'].fillna(df['price'].mean(), inplace=True)

# Mode for categorical columns (number_of_doors, fuel_type).
df['number_of_doors'].fillna(df['number_of_doors'].mode()[0], inplace=True)
df['fuel_type'].fillna(df['fuel_type'].mode()[0], inplace=True)

# Drop records if too many values are missing.
df_cleaned = df.dropna(thresh=2)

normalized_losses    0
bore                 0
stroke               0
horsepower           0
peak_rpm             0
price                0
dtype: int64


In [90]:
# 2. String & Categorical Data Preprocessing

# Convert categorical data into lowercase and remove special characters (make,
# fuel_type, aspiration, etc.).
categorical_columns = ['make', 'fuel_type', 'aspiration', 'number_of_doors', 'body_style', 'drive_wheels', 'engine_location', 'fuel_system']
for col in categorical_columns:
    df[col] = df[col].str.lower()
    df[col] = df[col].str.replace(r'[^a-zA-Z0-9\s]', '', regex=True)

# Replace "?" or NaN values with appropriate replacements.
for col in categorical_columns:
    df[col] = df[col].replace('?',np.nan)
    df[col] = df[col].fillna(df[col].mode()[0])
    
# Standardize number_of_cylinders:
# Convert words ("four", "six", "eight") to integers (4, 6, 8).
df['number_of_cylinders'] = df['number_of_cylinders'].replace({
    'four' : 4,
    'six' : 6,
    'five' : 5,
    'three' : 3,
    'twelve' : 12,
    'two' : 2,
    'eight' : 8
})
print(df['number_of_cylinders'])

# Standardize fuel_system categories (e.g., mpfi, spdi → Multi-Point Fuel Injection,
# Single-Point Fuel Injection).
df['fuel_system'] = df['fuel_system'].replace({
    'mpfi' : 'Multi-Point Fuel Injection',
    '2bbl' : 'Two Barrel',
    'mfi' : 'Multi-Point Fuel Injection ',
    '1bbl' : 'One Barrel',
    'spfi' : 'Single-Point Fuel Injection',
    '4bbl' : 'Four Barrel',
    'idi' : 'Indirect Injection',
    'spdi' : 'Single-Point Fuel Injection'
})
print(df['fuel_system'])

0      4
1      4
2      6
3      4
4      5
      ..
196    4
197    4
198    6
199    6
200    4
Name: number_of_cylinders, Length: 201, dtype: int64
0      Multi-Point Fuel Injection
1      Multi-Point Fuel Injection
2      Multi-Point Fuel Injection
3      Multi-Point Fuel Injection
4      Multi-Point Fuel Injection
                  ...            
196    Multi-Point Fuel Injection
197    Multi-Point Fuel Injection
198    Multi-Point Fuel Injection
199            Indirect Injection
200    Multi-Point Fuel Injection
Name: fuel_system, Length: 201, dtype: object


In [91]:
# 3. Numeric Feature Processing

# Convert "horsepower" and "peak_rpm" to integers (handle missing values first).
df['horsepower'] = pd.to_numeric(df['horsepower'], errors='coerce')
df['peak_rpm'] = pd.to_numeric(df['peak_rpm'], errors='coerce')

# Convert price to numeric and fill missing values.
df['price'] = pd.to_numeric(df['price'], errors='coerce')
df['price'].fillna(df['price'].median(), inplace=True)

# Standardize units (e.g., convert horsepower to kW if needed).
df['horsepower'] = df['horsepower'].astype(int)
df['peak_rpm'] = df['peak_rpm'].astype(int)
df['horsepower_kW'] = df['horsepower'] * 0.7457
print("Data after conversion and filling missing values:")
print(df)

Data after conversion and filling missing values:
     symboling  normalized_losses        make fuel_type aspiration  \
0            3                168  alfaromero       gas        std   
1            3                168  alfaromero       gas        std   
2            1                168  alfaromero       gas        std   
3            2                164        audi       gas        std   
4            2                164        audi       gas        std   
..         ...                ...         ...       ...        ...   
196         -1                 95       volvo       gas        std   
197         -1                 95       volvo       gas      turbo   
198         -1                 95       volvo       gas        std   
199         -1                 95       volvo    diesel      turbo   
200         -1                 95       volvo       gas      turbo   

    number_of_doors   body_style drive_wheels engine_location  wheel_base  \
0               two  convertible

In [92]:
# 4. Feature Engineering on Numeric Columns
# Create New Features:
# Power-to-weight ratio = horsepower / curb_weight.
df['power_to_weight'] = df['horsepower'] / df['curb_weight']

# Engine efficiency = horsepower / engine_size.
df['engine_efficiency'] = df['horsepower'] / df['engine_size']

# Fuel efficiency = city_mpg / highway_mpg.
df['fuel_efficiency'] = df['city_mpg'] / df['highway_mpg']

# Convert compression_ratio into categories:
# Low (<9), Medium (9-11), High (>11).
def categorize_compression_ratio(value):
    if value < 9:
        return 'Low'
    elif 9 <= value <= 11:
        return 'Medium'
    else:
        return 'High'
    
df['compression_ratio_category'] = df['compression_ratio'].apply(categorize_compression_ratio)
print(df)

     symboling  normalized_losses        make fuel_type aspiration  \
0            3                168  alfaromero       gas        std   
1            3                168  alfaromero       gas        std   
2            1                168  alfaromero       gas        std   
3            2                164        audi       gas        std   
4            2                164        audi       gas        std   
..         ...                ...         ...       ...        ...   
196         -1                 95       volvo       gas        std   
197         -1                 95       volvo       gas      turbo   
198         -1                 95       volvo       gas        std   
199         -1                 95       volvo    diesel      turbo   
200         -1                 95       volvo       gas      turbo   

    number_of_doors   body_style drive_wheels engine_location  wheel_base  \
0               two  convertible          rwd           front        88.6   
1    

In [93]:
# 5. Handling Date or Range Values
# Extract ranges from normalized_losses and create bins (e.g., Low, Medium, High
# loss categories).
df['normalized_losses'].fillna(df['normalized_losses'].median(), inplace=True)
bins = [0, 100, 200, np.inf]  
labels = ['Low', 'Medium', 'High'] 
df['loss_category'] = pd.cut(df['normalized_losses'], bins=bins, labels=labels, right=False)
print(df)

     symboling  normalized_losses        make fuel_type aspiration  \
0            3                168  alfaromero       gas        std   
1            3                168  alfaromero       gas        std   
2            1                168  alfaromero       gas        std   
3            2                164        audi       gas        std   
4            2                164        audi       gas        std   
..         ...                ...         ...       ...        ...   
196         -1                 95       volvo       gas        std   
197         -1                 95       volvo       gas      turbo   
198         -1                 95       volvo       gas        std   
199         -1                 95       volvo    diesel      turbo   
200         -1                 95       volvo       gas      turbo   

    number_of_doors   body_style drive_wheels engine_location  wheel_base  \
0               two  convertible          rwd           front        88.6   
1    

In [94]:
# 6. Encoding Categorical Features
# Label Encoding for fuel_type, aspiration, drive_wheels, etc.
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
categorical_columns = ['fuel_type', 'aspiration', 'drive_wheels', 'body_style', 'engine_type', 'fuel_system']

# One-Hot Encoding for body_style, engine_type, and fuel_system.
df['fuel_type'] = label_encoder.fit_transform(df['fuel_type'])
df['aspiration'] = label_encoder.fit_transform(df['aspiration'])
df['drive_wheels'] = label_encoder.fit_transform(df['drive_wheels'])

df_one_hot = pd.get_dummies(df[['body_style', 'engine_type', 'fuel_system']], drop_first=True)
df_encoded = pd.concat([df, df_one_hot], axis=1)
print(df_encoded)

     symboling  normalized_losses        make  fuel_type  aspiration  \
0            3                168  alfaromero          1           0   
1            3                168  alfaromero          1           0   
2            1                168  alfaromero          1           0   
3            2                164        audi          1           0   
4            2                164        audi          1           0   
..         ...                ...         ...        ...         ...   
196         -1                 95       volvo          1           0   
197         -1                 95       volvo          1           1   
198         -1                 95       volvo          1           0   
199         -1                 95       volvo          0           1   
200         -1                 95       volvo          1           1   

    number_of_doors   body_style  drive_wheels engine_location  wheel_base  \
0               two  convertible             2           

In [95]:
# 7. Data Scaling & Normalization
# Scale numerical features (length, width, height, curb_weight, engine_size)
# using Min-Max Scaling or Standardization.
from sklearn.preprocessing import MinMaxScaler, StandardScaler

min_max_scaler = MinMaxScaler()
standard_scaler = StandardScaler()

numerical_columns = ['length', 'width', 'height', 'curb_weight', 'engine_size']

df_min_max_scaled = df.copy()  # Make a copy to preserve original data
df_min_max_scaled[numerical_columns] = min_max_scaler.fit_transform(df[numerical_columns])

df_standardized = df.copy()  # Make a copy to preserve original data
df_standardized[numerical_columns] = standard_scaler.fit_transform(df[numerical_columns])

print("Min-Max Scaled Data:")
print(df_min_max_scaled)

print("\nStandardized Data:")
print(df_standardized)

Min-Max Scaled Data:
     symboling  normalized_losses        make  fuel_type  aspiration  \
0            3                168  alfaromero          1           0   
1            3                168  alfaromero          1           0   
2            1                168  alfaromero          1           0   
3            2                164        audi          1           0   
4            2                164        audi          1           0   
..         ...                ...         ...        ...         ...   
196         -1                 95       volvo          1           0   
197         -1                 95       volvo          1           1   
198         -1                 95       volvo          1           0   
199         -1                 95       volvo          0           1   
200         -1                 95       volvo          1           1   

    number_of_doors   body_style  drive_wheels engine_location  wheel_base  \
0               two  convertible    